In [2]:
import os
import sys

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import ToPILImage
from tqdm import tqdm
from training.dataset.transforms import ComposeAPI, NormalizeAPI

project_dir = os.path.dirname(os.getcwd())
print(project_dir)
sys.path.append(project_dir)

from mini_dataset import MiniDataset
from collate_fn import collate_fn

/home/guests/tuna_gurbuz/prototype/custom_models


In [3]:
len_video = 1
input_image_size = 512
transforms = [ComposeAPI([NormalizeAPI(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0], v2=True)])]
object_labels = [0]
batch_size = 1
shuffle = True
collate_fn = collate_fn
train_dataset = MiniDataset('train', len_video, input_image_size, object_labels, transforms, collate_fn, batch_size, shuffle, get_seg_mask=True)

Loading the take 001_PKA!



100%|██████████| 3/3 [00:11<00:00,  3.94s/it]


Loading the take 003_TKA!



100%|██████████| 3/3 [00:10<00:00,  3.46s/it]


Loading the take 005_TKA!



...

100%|██████████| 3/3 [00:03<00:00,  1.09s/it]


Loading the take 035_PKA!



100%|██████████| 3/3 [00:05<00:00,  1.98s/it]


Loading the take 037_TKA!



100%|██████████| 3/3 [00:11<00:00,  3.92s/it]


In [4]:
nb_samples = 0
mean = 0.0
std = 0.0
loader = train_dataset.get_loader()
for idx, (video, seg_mask) in enumerate(loader):
    img = video.img_batch.squeeze(0)
    batch_size, num_channels, height, width = img.shape
    im_vec = img.view(batch_size, img.size(1), -1)
    mean += im_vec.mean(2).sum(0)
    std += im_vec.std(2).sum(0)
    nb_samples += batch_size
    if idx == 10000:
        break

mean /= nb_samples
std /= nb_samples

print(f'Sample size: {nb_samples}')
print(f'Mean: {mean}')
print(f'Std: {std}')

10001
tensor([0.3578, 0.3607, 0.3501])
tensor([0.2861, 0.2749, 0.2712])
